In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


この辺を参考にしながら、コードを書いていく：
https://www.kaggle.com/code/samuelcortinhas/spaceship-titanic-a-complete-guide

必要そうなライブラリをインポートする（随時加筆）

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style = 'darkgrid', font_scale = 1.4)

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

`sns.histplot` あたりで、PandasとSeabornのバージョンの互換性に関する警告（将来のバージョンで、非推奨となる機能が含まれている）が出る。上記では、警告を無視する設定を追加する。

モデルを使用するため、いくつかimportする

In [ ]:
from sklearn.linear_mode import LinearRegression, LogsticRegression

### データの全体感の確認

In [ ]:
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

# (行数, 列数)
print('train.shape:', train.shape)
print('test.shape:', test.shape)

In [ ]:
train.head()

### 欠損値の確認

In [ ]:
# trainの欠損値
train.isna().sum()

In [ ]:
# testの欠損値
test.isna().sum()

### 重複行の確認

In [ ]:
train.duplicated().sum()

In [ ]:
test.duplicated().sum()

### 特徴量について

各種の特徴量を、以下の3つに分類する：

- continuous
- categorical
- descriptive / qualitative

| 特徴量|aaa |
|---|--- |
| PassengerId | descriptive / qualitative |
| HomePlanet |categorical |
| CryoSleep |categorical |
| Cabin | descriptive / qualitative |
| Destination |categorical |
| Age |continuous |
| VIP | categorical |
| RoomService |continuous |
| FoodCourt |continuous |
| ShoppingMail |continuous |
| Spa | continuous|
| VRDeck |continuous |
| Name |descriptive / qualitative |

In [ ]:
# datatypeの確認
train.dtypes

### 探索的データ解析

EDA（Exploratory Data Analysis）

In [ ]:
# 図のサイズ
plt.figure(figsize=(6, 6))

train['Transported'].value_counts().plot.pie(
    autopct = '%1.1f%%'
).set_title("Target distribution")

上図から、半分半分ということがわかる

乗客の年齢分布について視覚化する

In [ ]:
plt.figure(figsize = (10, 4))

sns.histplot(data=train, x ='Age', hue='Transported', binwidth=1, kde = True)

plt.title('Age distribution')
plt.xlabel('Age (years)')

0-18, 18-25, 25以上で、傾向が読み取れる

RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
→この辺は、金銭系なので、それで、まとめて傾向を見る
→傾向が似ていれば、1つの特徴量を作れる

...後で加筆するかも

Categorical featuresについて

- 傾向あり
- 特徴量から落とすこともできる

### Feature engineering

#### 年齢について

新しい特徴量である `Age_group` を定義する

In [ ]:
# train
# Age_groupを定義し、空文字で初期化する
# np.nanによる初期化だと、データタイプの不整合が起きる
train['Age_group'] = ''

train.loc[train['Age'] <= 12, 'Age_group'] = 'Age_0-12'
train.loc[(train['Age'] > 12) & (train['Age'] < 18), 'Age_group'] = 'Age_13-17'
train.loc[(train['Age'] >= 18 ) & (train['Age'] <= 25), 'Age_group'] = 'Age_18-25'
train.loc[(train['Age'] > 25 ) & (train['Age'] <= 30), 'Age_group'] = 'Age_26-30'
train.loc[(train['Age'] >= 30 ) & (train['Age'] <= 50), 'Age_group'] = 'Age_31-50'
train.loc[train['Age'] > 50, 'Age_group'] = 'Age_51+'

In [ ]:
# test
test['Age_group'] = ''

test.loc[test['Age'] <= 12, 'Age_group'] = 'Age_0-12'
test.loc[(test['Age'] > 12) & (test['Age'] < 18), 'Age_group'] = 'Age_13-17'
test.loc[(test['Age'] >= 18 ) & (test['Age'] <= 25), 'Age_group'] = 'Age_18-25'
test.loc[(test['Age'] > 25 ) & (test['Age'] <= 30), 'Age_group'] = 'Age_26-30'
test.loc[(test['Age'] >= 30 ) & (test['Age'] <= 50), 'Age_group'] = 'Age_31-50'
test.loc[test['Age'] > 50, 'Age_group'] = 'Age_51+'

In [ ]:
# 作成した特徴量に対して、グラフ化する
plt.figure(figsize=(10, 4))
g = sns.countplot(
    data = train,
    x = 'Age_group',
    hue = 'Transported',
    order = [
        'Age_0-12',
        'Age_13-17',
        'Age_18-25',
        'Age_26-30',
        'Age_31-50',
        'Age_51+'
    ]
)
plt.title('Age group distribution')

Age_0-12 については、transportedされている数が、そうでないのに比べ、多い

### どのモデルを使うか

いくつかある